In [31]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [16]:
file_list = [f"2020-1-{j}_df_all_features.csv" for j in [i for i in range(1,2)]]
df = pd.read_csv("2019-12-31_df_all_features.csv")
for k in file_list:
    df.append(pd.read_csv(k))

In [25]:
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [29]:
clf = LogisticRegression(random_state=0, solver = "lbfgs").fit(X, y)

In [30]:
clf.score(X, y)

0.7142857142857143